In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
traindata=pd.read_csv("../input/labeledTrainData.tsv",header=0,delimiter="\t",quoting=3)

In [ ]:
traindata.review.head()

In [ ]:
traindata.review[0]

In [ ]:
traindata.review[1]

In [ ]:
## to remove html markups we use beautiful soup
from bs4 import BeautifulSoup
example=BeautifulSoup(traindata.review[0])

In [ ]:
example

In [ ]:
example.get_text()

In [ ]:
traindata.review[0]

In [ ]:
#Dealing with punctuations,stopwords etc

In [ ]:
import re
re.sub("[^a-zA-Z]"," ",example.get_text())

In [ ]:
# all backslash and forward slash values are removed
# ^ means not present in 

In [ ]:
text1=re.sub("[^a-zA-Z]"," ",example.get_text())

In [ ]:
text1=text1.lower()    # important preprocessing steps
text1

In [ ]:
# remove stopwords
import nltk
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stpwords=stopwords.words('english')

In [ ]:
stpwords

In [ ]:
#Removing stopwords

In [ ]:
#code to work on training examples

In [ ]:
def review_to_words(raw_review):
    review_text=BeautifulSoup(raw_review).get_text()
    regex_review=re.sub("[^a-zA-Z]"," ",review_text)
    lower_review=regex_review.lower().split()
    list_review=[w for w in lower_review if w not in stpwords]
    return " ".join(list_review)


In [ ]:
%time
clean_review=review_to_words(traindata.review[0])

In [ ]:
def review_to_words(raw_review):
    review_text=BeautifulSoup(raw_review).get_text()
    regex_review=re.sub("[^a-zA-Z]"," ",review_text)
    lower_review=regex_review.lower().split()
    stpwords=set(stopwords.words('english'))
    list_review=[w for w in lower_review if w not in stpwords]
    return " ".join(list_review)


In [ ]:
%time
clean_review=review_to_words(traindata.review[0])

In [ ]:
traindata.review[0]

In [ ]:
#Two elements here are new: First, we converted the stop word list to a different data type, 
#a set. This is for speed; since we'll be calling this function tens of thousands of times, 
# it needs to be fast, and searching sets in Python is much faster than searching lists.

In [ ]:
num_reviews=len(traindata)

In [ ]:
num_reviews

In [ ]:
cleaned_train_reviews=[]
for i in range(num_reviews):
    cleaned_train_reviews.append(review_to_words(traindata.review[i]))

In [ ]:
cleaned_train_reviews[3]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer=CountVectorizer(analyzer='word',max_features=8000,ngram_range=(1,2))

In [ ]:
traindata_features=vectorizer.fit_transform(cleaned_train_reviews)

In [ ]:
train_data_features=traindata_features.toarray()

In [ ]:
train_data_features.shape

In [ ]:
df=pd.DataFrame(data=train_data_features,columns=vectorizer.get_feature_names())

In [ ]:
df.head()

In [ ]:
df.iloc[0][df.iloc[0] !=0]

In [ ]:
cleaned_train_reviews[0]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest=RandomForestClassifier(n_estimators=200)
forest=forest.fit(train_data_features,traindata.sentiment)

In [ ]:
test=pd.read_csv("../input/testData.tsv",header=0,delimiter='\t',quoting=3)


In [ ]:
test.head()

In [ ]:
num_test=len(test)

In [ ]:
cleaned_test_Reviews=[]
for i in range(num_test):
    cleaned_test_Reviews.append(review_to_words(test.review[i]))
    

In [ ]:
test_data_features=vectorizer.transform(cleaned_test_Reviews)

In [ ]:
test_df=pd.DataFrame(data=test_data_features.toarray(),columns=[vectorizer.get_feature_names()])

In [ ]:
test_df.head()

In [ ]:
result=forest.predict(test_data_features)

In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

In [ ]:
output

In [ ]:
output.to_csv("submission.csv", index = False, quoting = 3)